In [1]:
import mdtraj

from grand import lig_samplers as ls
from grand import lig_utils as lu

import openmm
import openmm.app
import openmm.unit
from pdbfixer import PDBFixer

import openff.toolkit

from openmmtools.integrators import VelocityVerletIntegrator

import numpy as np
import os
from collections import defaultdict

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
mol = openff.toolkit.Molecule.from_smiles("c1ccccc1O")
mol.generate_conformers(n_conformers=1000)
topology = mol.to_topology().to_openmm()
positions = mol.conformers[0].to_openmm()

In [3]:
fixer = PDBFixer(filename='1l56.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(False)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
openmm.app.PDBFile.writeFile(fixer.topology, fixer.positions, open('apo.pdb', 'w'))

In [4]:
prot = openmm.app.PDBFile('apo.pdb')

In [5]:
if os.path.exists('gcmc-ghosts.pdb'):
    pdb = openmm.app.PDBFile('gcmc-ghosts.pdb')
    sys_top = pdb.topology
    sys_pos = pdb.positions
else:
    sys_top, sys_pos, ghosts = lu.add_ghosts(prot.topology, prot.positions, topology, positions, n=500)
    openmm.app.PDBFile.writeFile(sys_top,sys_pos,'gcmc-ghosts.pdb')

In [6]:
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml','phenol_openff.xml')

In [7]:
prot_system = ff.createSystem(prot.topology)
#integrator = openmm.LangevinIntegrator(
#    298.0 * openmm.unit.kelvin, 1.0 / openmm.unit.picosecond, 2.0 * openmm.unit.femtosecond
#)
integrator = openmm.VerletIntegrator(
    1.0 * openmm.unit.femtosecond
)
simulation = openmm.app.Simulation(prot.topology,prot_system,integrator)

In [8]:
pdb = openmm.app.PDBFile('gcmc-ghosts.pdb')

In [9]:
pdb_system = ff.createSystem(pdb.topology)
integrator = openmm.LangevinIntegrator(
    298.0 * openmm.unit.kelvin, 1.0 / openmm.unit.picosecond, 2.0 * openmm.unit.femtosecond
)

In [10]:
frag_res_ids = []
for resid, residue in enumerate(pdb.topology.residues()):
    if residue.name == "UNK":
        frag_res_ids.append(resid)

In [11]:
sampler = ls.BaseGCMCSampler(pdb_system,pdb.topology,298*openmm.unit.kelvin,frag_res_ids,overwrite=True)

In [12]:
reporter = openmm.app.DCDReporter("phenol_sacp.dcd", reportInterval=100)

In [13]:
sampler.initialize(100, pdb.positions,integrator,reporter)

Initial Energy: 39459.14438056946 kJ/mol


In [14]:
f = open('real_res_ids.txt','w')
for n in range(100):
    sampler.move()
    sampler.reporter.report(sampler.simulation,sampler.context.getState(getPositions=True,getEnergy=True))
    f.write(','.join(sampler.real_lig_res_ids)+'\n')
    print(sampler.N)

0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.14438056946 kJ/mol
nan kJ/mol
0
39459.

KeyboardInterrupt: 